# Les espaces de noms

---

> Comme dans la plupart des langages "modernes", Rust utilise la notion de namespace. La syntaxe est empruntée de celle du C++ et ne présente guère de mauvaise surprise dès que l'on en a compris les principes généraux.

> Le séparateur entre les différents noms est **`::`**.

## La notion de crate

**Votre programme** que vous gênerez avec la commande cargo new **est une crate**. Une crate est une unité de compilation en Rust qui peut être composée d'un ou de plusieurs fichiers. Si maintenant vous gênerez une librairie pour votre programme avec cargo new --lib, **cette librairie sera aussi une crate**. Et enfin, si vous importez la librairie colored depuis crates.io, votre projet aura **la crate colored** en plus ainsi que toutes ses dépendances, **qui sont aussi des crates**.

> Pour résumer, une crate est soit un programme compose de la fonction main, soit une librairie.

### Utiliser sa propre librairie

Admettons que vous ayez compilé votre programme ainsi qu'une librairie de votre cru, le fichier **cargo.toml** de votre programme principal contiendra le chemin de cette librairie et le nom du namespace associé qui est le nom de la librairie avec la configuration par défaut.

Supposons que votre librairie se nomme **ma_lib**, et qu'à l'intérieur vous ayez défini la fonction **ma_fonction** qui retourne un entier :

Afin de pouvoir utiliser ma_fonction depuis votre programme principal, le chemin associe sera `ma_lib::ma_fonction`. Plusieurs possibilités s'offrent à vous pour utiliser cette fonction :
- L'expression du chemin littéral.
- L'utilisation de `use`.

>```
    // Chemin litteral.
    let a = ma_lib::ma_fonction();
>```

>```
    // Importation de la fonction dans le namespace du programme principal.
    use ma_lib::ma_fonction;
    // ma_fonction() est utilisable comme si la fonction etait definie ici.
    let a = ma_fonction();
>```

>```
    // Importation de tous les noms public de la librairie.
    use ma_lib::*;
    // ma_fonction() fait partie d'un se ces noms.
    let a = ma_fonction();
>```

*ma_fonction sera déclarée dans la lib en tant que public via le préfixe `pub`.*

### Utiliser une crate de crates.io

Supposons maintenant que vous voulez colorier en cyan les chaînes de sortie de votre programme, vous allez dans votre fichier cargo.toml ajouter aux dépendances la crate colored. À la première compilation du programme, cette crate ainsi que toutes ses dépendances sera telechargée puis compilée.

Afin de pouvoir colorier votre texte en cyan, vous irez importer le trait Colorize de la crate colored (oui, un trait, nous verrons plus tard ce que c'est), un trait qui vous permettra d'utiliser une fonction cyan() sur vos chaînes de caractère :

>```
    use colored::Colorize;
    println!("{}", "text colorized !".cyan());
>```

Sachez que bien que très peu pratique dans ce cas précis, la forme littérale était aussi possible :

>```
    println!("{}", colored::Colorize::cyan("Hello, world!"));
>```

Ainsi que celle avec le wildcard :

>```
    use colored::*;
    println!("{}", "text colorized !".cyan());
>```

*Pour plus d'information sur comment installer la crate colored, voir le chapitre 13 - 001 Using Crates.* 

### Le `use crate`

- Pour faire des tests unitaires, le code que vous écrirez ressemblera à cela :

In [ ]:
#[cfg(test)]  // directive de precompilation, ne compile le bloc que par `cargo test`
mod add_one_test { // sous-module de la crate principale
    use crate::add_one; // importe la fonction add_one depuis le main.rs ou le lib.rs

    #[test]
    fn check_add_one() {
        assert_eq!(add_one(1), 2)
    }
}

> Le mot-clef `mod` indique que nous sommes dans un sous-module. Dans ce module, les noms des items du fichier main.rs ne sont plus disponibles, alors il nous faut importer ce dont on a besoin grâce a l'instruction. `use crate::add_one`. crate ici fait référence à la racine du projet, le fichier main.rs ou lib.rs pour une librairie.

> Si nous avions voulu importer plusieurs noms, nous aurions pu écrire :
> - `use crate::*;` pour importer la totalité des noms du fichier main.rs.
> - `use crate::{add_one, add_two, une_troisieme_fonction};` pour n'importer qu'une liste de noms grâce aux brackets `{...}`.
> - Ne pas faire de `use` et écrire directement `assert_eq!(crate::add_one(1), 2)` !

### Le mot clef `super`

- Enfin, il existe le mot-clef `super` qui est une référence vers le namespace du parent :

In [4]:
fn droopy() {
    println!("You know what ? I'm happy.");
}

fn main() {
    song::droopy_speack();
}

mod song {
    use super::droopy; // Importe la fonction du namespace parent (ici le namespace principal)
    pub fn droopy_speack() { // rien a voir mais pub ici est obligatoire pour etre appele depuis d'autres namespaces.
        droopy();
    }
}
main()

You know what ? I'm happy.


()

**NB : Nous avons dans ces exemples importé une fonction, mais il est possible d'importer toutes sortes d'éléments telles des structures, des énumérations ou les traits.**

## Les imports depuis la std

- Pour ce qui est commun d'utiliser tels les Option, les Result ou quelques structures d'allocation dynamique, les développeurs de Rust ont fait ce que l'on appelle le `prelude`. Les éléments présents dans le prelude Rust sont directement importes dans tous nos fichiers et il n'est pas nécessaire d'utiliser des use ou de donner leurs chemins.

In [8]:
// Option, Some et Box sont dans le prelude de Rust ...

let opt: Option<usize> = Some(23); // Option n'a pas besoin d'etre importe
let opt: std::option::Option<usize> = std::option::Option::Some(24); // Vraiment, ca ne sert a rien !

let b: Box<usize> = Box::new(12); // Idem pour Box
let b: std::boxed::Box<usize> = std::boxed::Box::new(12); // Woot ?

- Mais il arrive que ce ne soit pas toujours le cas :

In [9]:
{
    let size = size_of::<usize>();
}

Error: cannot find function `size_of` in this scope

- Le namespace doit donc être précisé ici :

In [10]:
let size = std::mem::size_of::<usize>();

## Les alias

- Il est possible de renommer grace au mot clef `as` les noms. Que ce soit pour éviter d'éventuels conflits ou bien parce que le nouveau nom inspire davantage le développeur, il a toujours la possibilité de le faire :

In [12]:
use std::mem::size_of as get_the_type_size // name alias
let size = get_the_type_size::<usize>();